<a href="https://colab.research.google.com/github/wutchy/national-flag/blob/master/national_flag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.u.tsukuba.ac.jp/~s1711435/national_flags/Japan.jpg

In [ ]:
# パッケージインストール

!pip install beautifulsoup4
!pip install icrawler
!pip install janome

In [ ]:
# スクレイピング
import requests
from bs4 import BeautifulSoup

target_url = 'https://www.mofa.go.jp/region/asia-paci/index.html'
r = requests.get(target_url)
soup = BeautifulSoup(r.text, 'html.parser')

asia_countries = []

divs = soup.find("div", id="area01").find_all("div")
for div in divs:
  lis = div.ul.find_all("li", class_="styled2")
  for li in lis:
    asia_countries.append(li.a.string)

asia_countries.append('Japan')

print(asia_countries)


In [ ]:
# キーワードを指定し、画像検索し、画像をダウンロード
import sys
import os


#keywords = ['red circle', 'yellow circle']
keywords = ['the United States of America national flag', 'Croatia']

download(keywords, skip=True)
img_to_pkl(keywords, skip=True)

# file_path = os.path.abspath("./images/palau_national_flag/000001.jpg")

# res1 = red_circle_estimate.has_red_circle(file_path)
# res2 = white_background_estimate.has_white_background(file_path)

In [ ]:
# 画像検索し、ダウンロード
from icrawler.builtin import BingImageCrawler
import os


def download(args, skip=False):

    for arg in args:
        save_dir = f"/content/drive/My Drive/national_flag/images/flags/{arg.replace(' ', '_')}"
        if not os.path.isdir(save_dir):
            os.mkdir(save_dir)

        if not (skip is True and len(os.listdir(save_dir)) > 0):
            crawler = BingImageCrawler(storage={'root_dir': save_dir})
            crawler.crawl(keyword=arg, max_num=1)


In [ ]:
# ディレクトリ内の画像のピックル化
import os
from os import path
import glob
from PIL import Image, ImageFilter
import matplotlib.pyplot as plt
import numpy as np
import pickle


def img_to_pkl(keywords, skip=False):
    img_size = (200, 200)

    for keyword in keywords:
        if not os.path.exists(f"{keyword.replace(' ', '_')}_imgs.pkl"):
            first_file = True
            files = glob.glob(f"/content/drive/My Drive/national_flag/images/flags/{keyword.replace(' ', '_')}/*")
 
            for f in files:
                img = Image.open(f)
                img = img.resize(img_size)
                img = img.convert("RGB")
                img = np.array(img).reshape(1, 200, 200, 3)
                if first_file:
                    imgs = img
                    first_file = False
                else:
                    imgs = np.append(imgs, img, axis=0)

            with open(f"/content/drive/My Drive/national_flag/{keyword.replace(' ', '_')}_imgs.pkl", 'wb') as f:
                pickle.dump(imgs, f)


In [ ]:
# 全ての画像ディレクトリの中から、一枚ずつ画像を取得し、特定のディレクトリに保存
import os
import glob
from PIL import Image, ImageFilter
import matplotlib.pyplot as plt
import numpy as np
import pickle
import shutil


def get_each_dir_image(parent_dir_path):
  path = parent_dir_path
  files = os.listdir(path)
  dirs = [f for f in files if os.path.isdir(os.path.join(path, f))]

  for dir in dirs:
    copy_from = glob.glob(path + '/' + dir + '/000001*')[0]
    copy_to = os.path.dirname(path) + '/ycs_imgs'
    if not os.path.exists(copy_to):
      os.mkdir(copy_to)

    print(shutil.copy(copy_from, copy_to + '/' + dir + os.path.splitext(copy_from)[1]))


get_each_dir_image("/content/drive/My Drive/national_flag/images/flags")

In [ ]:
# ヤフークラウドソーシングのTSV作成
import glob
import copy
import os
import csv
import random
import pprint
from collections import Iterable

perms = []
files1 = list(map(os.path.basename, glob.glob("/content/drive/My Drive/national_flag/images/ycs_imgs/*")))
server_dir = "https://www.u.tsukuba.ac.jp/~s1711435/national_flags/"
files2 = copy.copy(files1)

header = ["設問ID(半角英数字20文字以内)",	"チェック設問有無(0:無 1:有)", "チェック設問の解答(F07用)", "F01:ラベル",	"F02:ラベル", "F03:画像（ＵＲＬ）", "F04:ラベル",	"F05:画像（ＵＲＬ）", "F06:ラベル", "F07:テキストエリア",	"F08:ラベル"]

f01 = "タスク内容####以下の2つの国旗の画像を見比べたとき、左の国旗の画像に関して、右の国旗の画像には無く、左の国旗の画像にはある特徴を、できるかぎり多く挙げてください。##(タスクの紹介画面の回答例を見てから問題に取り組んでください)####"
f02 = "画像１(特徴を挙げる画像)"
f04 = "画像２(比較する画像)"
f06 = "左の国旗の画像に関して、右の国旗の画像には無く、左の国旗の画像にはある特徴を、できるかぎり多く下の入力欄に入力してください"
f08 = "※注意事項##・左の国旗の画像にはあって、右の国旗の画像にはない特徴を入力してください##・小学生に説明できるような、簡潔かつ明瞭な回答をお願いいたします。##・できるだけ多くの区別可能な特徴を入力してください##・タスク指示文に従った十分な回答をしていただけない方は次回以降のタスクで回答不可リストに登録させていただくことがあります。逆に、誠実に取り組んでいただけた場合は今後も同様のタスクをお願いすることがあります。####いただいた結果は、学術研究に役立たせていただきます。引き続き作業をお願いいたします。"

for file1 in files1:
  for file2 in files2:
    if file1 != file2:
      perms.append([file1, file2])

random.shuffle(perms)

with open('/content/drive/My Drive/national_flag/tasks.tsv', 'w') as f:
    writer = csv.writer(f, delimiter='\t')
    writer.writerow(header)
    for i in range(len(perms)+3):
      if i < len(perms):
        writer.writerow([str(i+1), str(0), ' ', f01, f02, server_dir + perms[i][0], f04, server_dir + perms[i][1], f06, ' ', f08])
      else:
        perm = random.choice(perms)
        writer.writerow([str(i+1), str(0), ' ', f01, f02, server_dir + perm[0], f04, server_dir + perm[1], f06, ' ', f08])


In [ ]:
!pip install janome==0.3.10

     |████████████████████████████████| 21.5MB 1.4MB/s 
  Found existing installation: Janome 0.4.0
    Uninstalling Janome-0.4.0:
      Successfully uninstalled Janome-0.4.0


In [2]:
# TSVから回答を取得。

import matplotlib.pyplot as plt
import os
import glob
import numpy as np
import csv
from janome.tokenizer import Tokenizer
from collections import Iterable

t = Tokenizer()

countries = ['Bangladesh', 'Bhutan', 'Cambodia', 'China', 'India', 'Indonesia', 'Republic_of_Korea', 'Laos', 'Malaysia', 'Maldives', 'Mongolia', 'Myanmar', 'Nepal', 'Pakistan', 'Singapore', 'Sri_Lanka', 'Thailand', 'Timor-Leste', 'Japan', 'Viet_Nam', 'Philippines', 'Brunei']
# files1 = list(map(os.path.basename, glob.glob("/content/drive/My Drive/national_flag/images/ycs_imgs/*"))) 
# for file in files1:
#   countries.append(os.path.splitext(file)[0])
# print(countries)

def flatten(l):
    for el in l:
        if isinstance(el, Iterable) and not isinstance(el, basestring):
            for sub in flatten(el):
                yield sub
        else:
            yield el

with open('/content/drive/My Drive/national_flag/national_flag.tsv') as f:
    tsv = csv.reader(f, delimiter = '\t')
    data = [row for row in tsv]

features = []
country_features = {}
words = []

for row in data:
    c1 = row[4].rsplit('/', 1)[1].split('.')[0].strip('_')
    c2 = row[6].rsplit('/', 1)[1].split('.')[0].strip('_')
    features.append(row[10])
    if c1 not in country_features:
      country_features[c1] = {}

    if c2 not in country_features[c1]:
      country_features[c1][c2] = []

    country_features[c1][c2].append(row[10])
    



print(features)
print(countries)

print("country_features length: " + str(len(country_features["Japan"])))
print(country_features)

with open('/content/drive/My Drive/national_flag/country_x_feature.csv', 'w') as f:
    writer = csv.writer(f)

    for k1, v1 in country_features.items():
      for k2, v2 in v1.items():
        row = []
        row.append(k1)
        row.append(f"https://www.u.tsukuba.ac.jp/~s1711435/national_flags/{k1}.jpg")
        row.append(k2)
        row.append(f"https://www.u.tsukuba.ac.jp/~s1711435/national_flags/{k2}.jpg")

        words = []
        for feature in v2:
          row.append(feature)
          wakati_words = t.tokenize(feature, wakati=True)
          words.append(wakati_words)
        
        
        row.append(flatten(words))
        writer.writerow(row)
    








# labels = ["自動４輪車ではない", "自動４輪車である"]
# image_dir = "./car_task_imgs/"

# files = os.listdir(image_dir)

# t = Tokenizer()
# for reason in reasons:
#     wakati_words = t.tokenize(reason, wakati=True)
#     words.append(wakati_words)

# dup_words = sum(words,[])
# no_dup_words = list(set(dup_words))
# words = []

# for word in no_dup_words:
#     cnt = dup_words.count(word)
#     if cnt > 3:
#         words.append(word)


# selected_words = ['大き', '搭載', '席', '判別', '乗用', '側面', '後方', '古い', '車体', '自動', 'ダブル', '普通', '車種', 'SUV', '片方', '多', '積ん', 'セダン', '左右', '支える', '形', '面', '後部', 'ドア', '駆動', '前方', '長', 'トラック', 'スクーター', '大きく', 'タイヤ', '後ろ', 'バランス', '前後', '輪', '右側', 'バン', '重量', 'バイク', '四角', '運転', 'タイプ', '逆', '反対', '荷台', '後', '全体', '形態', '一般', '人', '乗用車', '型式', 'バス', '形状', '車輪', '荷物', 'タンク', '片側', '車', '車両', '大型', '高', '支え', '乗り', '左側', '小型', '個', 'ワゴン', '前輪', '自動車', '部分', '二輪車']


# word_freqs = []
# for word in selected_words:
#     cnt = 0
#     for reason in reasons:
#         if word in reason:
#             cnt += 1

#     word_freqs.append(cnt)


# word_hash = {}
# for i in range(len(selected_words)):
#     word_hash[selected_words[i]] = word_freqs[i]


# word_hash = sorted(word_hash.items(), key=lambda x:x[1], reverse=True)

# word_freqs = []
# selected_words = []
# for (k,v) in word_hash:
#     word_freqs.append(v)
#     selected_words.append(k)

# with open('words_x_reasons.csv', 'w') as f:
#     writer = csv.writer(f)
#     writer.writerow(['', ''] + selected_words)
#     writer.writerow(['画像リンク','頻度'] + word_freqs)

#     img_idx = 1
#     img_idx_cnt = 0
#     for i, reason in enumerate(reasons):
#         cols = []
#         cols.append(f"https://www.u.tsukuba.ac.jp/~s1711435/car_task_imgs/task{img_idx}.jpg")
#         img_idx_cnt += 1
#         if img_idx_cnt >= 5:
#             img_idx += 1
#             img_idx_cnt = 0

#         cols.append(reason)
#         for word in selected_words:
#             if word in reason:
#                 cols.append('✔︎')
#             else:
#                 cols.append('')

#         writer.writerow(cols)


['水色##黄色', '白い部分がある##横縞', '色が上下で二色に分かれている事、白い色が使われている事、といった特徴がある', '黄色##黒色##星', '丸い模様がある。##縞模様がある##青い部分がある', '真ん中に円が描かれている事、三色で構成されている事、青色が使われている事、といった特徴を持つ', 'オレンジ色##紋章', '三日月の模様##星の模様', '2色で構成されている事、月と星が描かれている事、直線が一切描かれていない事、といった特徴を持つ', '黒色##赤い紋章', '三日月の模様##星の模様##横縞', 'ストライプ模様が使用されている事、月と太陽が描かれている事、青色が使われている事、といった特徴を持つ', '緑色##月', '竜の絵##黄色い部分', '対角線にラインが描かれている事、中央に龍が描かれている事、黄色とオレンジが使われている事、といった特徴を持つ', '半月、太陽マーク、青、黄色', '縞模様がある\u3000カラフル\u3000星のような模様がある', '縞模様がある##黄色が使われている##青色が使われている##月がある##星がある', '中央の手のマーク、斜線、黒、黄色、赤', '多色、斜めに縞がある', '黄色が使われている##赤色が使われている##黒色が使われている##斜めの線がある', '〇、中央の赤', 'シンプルなデザイン', '中央に円がある', '半月、2重四角、緑、白', '三日月模様がある、三色', '緑色が使われている##月がある##中央のマークが四角で囲われている', 'シンプルに中央に〇、丸が赤', 'シンプルなデザイン、大きな丸', '中央の円が赤い', '緑', '星に色がついていない。背景が３色に分かれている。', 'あ', '水色', '色が縦三色に並んでいる。魚のような絵が記載されている。', 'か', '黄色\u3000龍', '竜のような絵が描いてある。斜めに２色に分かれている。', 'か', '緑\u3000月', '三日月のような絵面である。', 'か', '青', '幾何学的な模様で構成されている。', 'か', '色が複数ある。星形が挿入されている。', '☆、黄色、緑', '星がある。##色が負数ある。', 'ほぼ真ん中に丸がある。白抜きの模様ではない。', '赤